### Import Lib

In [ ]:
import os
import sys
import math
import yaml
import torch
import joblib
import warnings
from tqdm import tqdm
import torch.nn as nn

### Utility files

In [ ]:
def config_files():
    with open("../config.yml", "r") as config_file:
        return yaml.safe_load(config_file)


def dump_file(value: None, filename: None):
    if (value is not None) and (filename is not None):
        joblib.dump(value=value, filename=filename)
    else:
        print("Error: 'value' and 'filename' must be provided.".capitalize())


def load_file(filename: None):
    if filename is not None:
        return joblib.load(filename=filename)
    else:
        print("Error: 'filename' must be provided.".capitalize())


def weight_init(m):
    classname = m.__class__.__name__
    if classname.find("Linear") != -1:
        nn.init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)
    elif classname.find("Conv") != -1:
        nn.init.kaiming_normal_(m.weight.data)
        if m.bias is not None:
            nn.init.constant_(m.bias.data, 0.0)


def device_init(device: str = "cuda"):
    if device == "cuda":
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")
    elif device == "mps":
        return torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    else:
        return torch.device("cpu")

def clean_folders():
    TRAIN_MODELS: str = "../artifacts/checkpoints/train_models/"
    BEST_MODEL: str = "../artifacts/checkpoints/best_model/"
    METRICS_PATH: str = "../artifacts/metrics/"
    TRAIN_IMAGES: str = "../artifacts/outputs/train_images/"
    TEST_IMAGE: str = "../artifacts/outputs/test_image/"

    warnings.warn(
        "Warning! This will remove the previous files, which might be useful in the future. "
        "You may want to download the previous files or use MLflow to track and manage them. "
        "Suggestions for updating them are welcome."
    )

    for path in tqdm(
        [TRAIN_MODELS, BEST_MODEL, METRICS_PATH, TRAIN_IMAGES, TEST_IMAGE]
    ):
        for files in os.listdir(path):
            file_path = os.path.join(path, files)
            try:
                if os.path.isfile(file_path):
                    os.remove(file_path)
            except Exception as e:
                print(f"Error occurred while deleting {file_path}: {e}")

        print("{} folders completed".format().capitalize())

### Patch Embedding

In [ ]:
class PatchEmbedding(nn.Module):
    def __init__(
        self,
        channels: int = 3,
        patch_size: int = 16,
        image_size: int = 128,
        dimension: int = None,
    ):
        super(PatchEmbedding, self).__init__()

        self.in_channels = channels
        self.patch_size = patch_size
        self.dimension = dimension
        self.image_size = image_size

        if self.dimension is None:
            self.dimension = (self.patch_size**2) * self.in_channels

        self.number_of_pathches = (self.image_size // self.patch_size) ** 2

        self.kernel_size, self.stride_size = self.patch_size, self.patch_size
        self.padding_size = self.patch_size // self.patch_size

        self.projection = nn.Conv2d(
            in_channels=self.in_channels,
            out_channels=self.dimension,
            kernel_size=self.kernel_size,
            stride=self.stride_size,
            padding=self.padding_size,
            bias=False,
        )

        self.positional_embeddings = nn.Parameter(
            torch.randn(self.padding_size, self.number_of_pathches, self.dimension),
            requires_grad=True,
        )

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            x = self.projection(x)
            x = x.view(x.size(0), x.size(1), -1)
            x = x.permute(0, 2, 1)
            x = self.positional_embeddings + x
            return x
        else:
            raise ValueError("Input must be a torch tensor.".capitalize())

    @staticmethod
    def total_params(model):
        if isinstance(model, PatchEmbedding):
            return sum(params.numel() for params in model.parameters())


if __name__ == "__main__":
    image_channels = config_files()["patchEmbeddings"]["channels"]
    image_size = config_files()["patchEmbeddings"]["image_size"]
    patch_size = config_files()["patchEmbeddings"]["patch_size"]
    dimension = config_files()["patchEmbeddings"]["dimension"]

    pathEmbedding = PatchEmbedding(
        channels=image_channels,
        patch_size=patch_size,
        image_size=image_size,
        dimension=dimension,
    )

    image = torch.randn(
        (image_channels // image_channels, image_channels, image_size, image_size)
    )

    assert (pathEmbedding(image).size()) == (
        image_channels // image_channels,
        (image_size // patch_size) ** 2,
        dimension,
    )

### Scaled Dot Product -> self attention 

In [ ]:
def scaled_dot_product(query: torch.Tensor, key: torch.Tensor, values: torch.Tensor):
    if (
        isinstance(query, torch.Tensor)
        and isinstance(key, torch.Tensor)
        and isinstance(values, torch.Tensor)
    ):
        logits = torch.matmul(query, key.transpose(-1, -2)) / math.sqrt(query.size(-1))
        attention_weights = torch.softmax(logits, dim=-1)
        attention_output = torch.matmul(attention_weights, values)
        return attention_output

    else:
        raise ValueError("All inputs must be torch tensors.".capitalize())

if __name__ == "__main__":
    query = torch.randn((1, 8, 64, 32))
    key = torch.randn((1, 8, 64, 32))
    values = torch.randn((1, 8, 64, 32))

    assert (scaled_dot_product(query, key, values).size()) == (1, 8, 64, 32)

### Multi Head Attention Layer-> attention is all you need

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, nheads: int = 8, dimension: int = 256):
        super(MultiHeadAttentionLayer, self).__init__()
        self.nheads = nheads
        self.dimension = dimension

        warnings.warn(
            "Please ensure that the dimension is a multiple of the number of heads in the encoder block (e.g., 256 % 8 = 0). "
            "This is a requirement for the Transformer Encoder Block to function properly. "
            "If not, you might need to adjust the dimension or the number of heads."
        )
        assert (
            dimension % self.nheads == 0
        ), "Dimension mismatched with nheads and dimension".title()

        self.QKV = nn.Linear(
            in_features=self.dimension, out_features=3 * self.dimension, bias=False
        )

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            QKV = self.QKV(x)
            query, key, values = torch.chunk(input=QKV, chunks=3, dim=-1)
            query = query.view(
                query.size(0), query.size(1), self.nheads, self.dimension // self.nheads
            )
            key = key.view(
                key.size(0), key.size(1), self.nheads, self.dimension // self.nheads
            )
            values = values.view(
                values.size(0),
                values.size(1),
                self.nheads,
                self.dimension // self.nheads,
            )

            query = query.permute(0, 2, 1, 3)
            key = key.permute(0, 2, 1, 3)
            values = values.permute(0, 2, 1, 3)

            attention_output = scaled_dot_product(query=query, key=key, values=values)
            attention_output = attention_output.view(
                attention_output.size(0),
                attention_output.size(2),
                attention_output.size(1) * attention_output.size(-1),
            )

            return attention_output
        else:
            raise ValueError("Input must be a torch tensor.".capitalize())

    @staticmethod
    def total_params(model):
        if isinstance(model, TransformerEncoderBlock):
            return sum(params.numel() for params in model.parameters())


if __name__ == "__main__":
    nheads = config_files()["transfomerEncoderBlock"]["nheads"]
    dimension = config_files()["patchEmbeddings"]["dimension"]
    image_channels = config_files()["patchEmbeddings"]["channels"]
    image_size = config_files()["patchEmbeddings"]["image_size"]
    patch_size = config_files()["patchEmbeddings"]["patch_size"]
    dimension = config_files()["patchEmbeddings"]["dimension"]

    multihead_attention = MultiHeadAttentionLayer(
        nheads=nheads, dimension=dimension
    )

    image = torch.randn((1, (image_size // patch_size) ** 2, dimension))

    assert (multihead_attention(image).size()) == (
        (1, (image_size // patch_size) ** 2, dimension)
    )

### Feed Forward Neural Network

In [ ]:
class FeedForwardNeuralNetwork(nn.Module):
    def __init__(
        self,
        in_features: int = 256,
        out_features: int = 4 * 256,
        dropout: float = 0.1,
        activation: str = "relu",
    ):
        super(FeedForwardNeuralNetwork, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.activation = activation

        if activation == "relu":
            self.activation_function = nn.ReLU()
        elif activation == "gelu":
            self.activation_function = nn.GELU()
        elif activation == "selu":
            self.activation_function = nn.SELU(inplace=True)
        else:
            raise ValueError(
                "Invalid activation function. Choose from'relu', 'gelu', or'selu'."
            )

        self.layers = []

        for idx in range(2):
            self.layers.append(
                nn.Linear(in_features=self.in_features, out_features=self.out_features)
            )
            self.in_features = self.out_features
            self.out_features = in_features

            if idx == 0:
                self.layers.append(self.activation_function)
                self.layers.append(nn.Dropout(self.dropout))

        self.network = nn.Sequential(*self.layers)

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            return self.network(x)
        else:
            raise ValueError("Input must be a torch.Tensor.")


if __name__ == "__main__":
    network = FeedForwardNeuralNetwork(
        in_features=256,
        out_features=4 * 256,
        dropout=0.1,
        activation="relu",
    )

    print(network(torch.randn((1, 64, 256))).size())

### Layer Normalization

In [ ]:
class LayerNormalization(nn.Module):
    def __init__(self, dimension: int = 256, layer_norm_eps: float = 1e-5):
        super(LayerNormalization, self).__init__()
        self.dimension = dimension
        self.layer_norm_eps = layer_norm_eps

        self.alpha = nn.Parameter(
            data=torch.ones((1, 1, self.dimension)), requires_grad=True
        )
        self.beta = nn.Parameter(
            data=torch.zeros((1, 1, self.dimension)), requires_grad=True
        )

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            mean = torch.mean(x, dim=-1).unsqueeze(-1)
            variance = torch.var(x, dim=-1).unsqueeze(-1)

            y = (x - mean) / torch.sqrt(variance + self.layer_norm_eps)

            return self.alpha * y + self.beta

        else:
            raise ValueError("Input must be a torch.Tensor.".capitalize())


if __name__ == "__main__":
    layer_normalization = LayerNormalization(dimension=256)
    assert (layer_normalization(torch.randn((1, 64, 256))).size())

### Transformer Encoder Block

In [ ]:
class TransformerEncoderBlock(nn.Module):
    def __init__(
        self,
        dimension: int = 256,
        nheads: int = 8,
        dim_feedforward: int = 1024,
        dropout: float = 0.1,
        layer_norm_eps: float = 1e-5,
        activation: str = "relu",
    ):
        super(TransformerEncoderBlock, self).__init__()
        self.dimension = dimension
        self.nheads = nheads
        self.dim_feedforward = dim_feedforward
        self.activation = activation
        self.dropout = dropout
        self.layer_norm_eps = layer_norm_eps

        self.multihead_attention = MultiHeadAttentionLayer(
            nheads=self.nheads, dimension=self.dimension
        )
        self.layer_normalization = LayerNormalization(
            dimension=self.dimension, layer_norm_eps=self.layer_norm_eps
        )
        self.feed_forward_network = FeedForwardNeuralNetwork(
            in_features=self.dimension,
            out_features=4 * self.dim_feedforward,
            activation=self.activation,
            dropout=self.dropout,
        )
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            residual = x

            x = self.multihead_attention(x)
            x = self.dropout(x)
            x = torch.add(x, residual)
            x = self.layer_normalization(x)

            residual = x

            x = self.feed_forward_network(x)
            x = self.dropout(x)
            x = torch.add(x, residual)
            x = self.layer_normalization(x)

            return x
        else:
            raise ValueError("Input must be a torch.Tensor.")

    @staticmethod
    def total_params(model):
        if isinstance(model, TransformerEncoderBlock):
            return sum(params.numel() for params in model.parameters())

if __name__ == "__main__":
    nheads = config_files()["transfomerEncoderBlock"]["nheads"]
    dimension = config_files()["patchEmbeddings"]["dimension"]
    image_channels = config_files()["patchEmbeddings"]["channels"]
    image_size = config_files()["patchEmbeddings"]["image_size"]
    patch_size = config_files()["patchEmbeddings"]["patch_size"]
    dimension = config_files()["patchEmbeddings"]["dimension"]
    dropout = config_files()["transfomerEncoderBlock"]["dropout"]
    activation = config_files()["transfomerEncoderBlock"]["activation"]
    
    transformer_encoder_block = TransformerEncoderBlock(
        dimension=dimension,
        nheads=nheads,
        dim_feedforward=dimension,
        dropout=dropout,
        activation=activation,
        layer_norm_eps=1e-5,
    )
    
    print(transformer_encoder_block(torch.randn((1, (image_size // patch_size) ** 2, dimension))).size())

### Transformer Encoder -> attention is all you need

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(
        self,
        dimension: int = 256,
        nheads: int = 8,
        num_encoder_layers: int = 6,
        dim_feedforward: int = 1024,
        dropout: float = 0.0,
        layer_norm_eps: float = 1e-05,
        activation: str = "relu",
    ):
        super(TransformerEncoder, self).__init__()
        self.dimension = dimension
        self.nheads = nheads
        self.num_encoder_layers = num_encoder_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.layer_norm_eps = layer_norm_eps
        self.activation = activation

        self.layers = nn.Sequential(
            *[
                TransformerEncoderBlock(
                    dimension=self.dimension,
                    nheads=self.nheads,
                    dim_feedforward=self.dim_feedforward,
                    dropout=self.dropout,
                    activation=self.activation,
                )
                for _ in tqdm(range(self.num_encoder_layers))
            ]
        )

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            for layer in self.layers:
                x = layer(x=x)
            return x
        else:
            raise ValueError("Input must be a torch.Tensor.".capitalize())


if __name__ == "__main__":
    transformerEncoder = TransformerEncoder(
        dimension=256,
        nheads=8,
        num_encoder_layers=8,
        dim_feedforward=1024,
        dropout=0.1,
        layer_norm_eps=1e-05,
        activation="relu",
    )

    print(transformerEncoder(torch.randn((1, 64, 256))).size())

### Loss Function : BCELoss

In [ ]:
class LossFunction(nn.Module):
    def __init__(self, loss_name: str = "BCEWithLogitsLoss", reduction: str = "mean"):
        super(LossFunction, self).__init__()
        self.loss_name = loss_name
        self.reduction = reduction

        if loss_name == "BCEWithLogitsLoss":
            self.criterion = nn.BCEWithLogitsLoss(reduction=reduction)
        elif loss_name == "BCELoss":
            self.criterion = nn.BCELoss(reduction=reduction)
        elif loss_name == "CCE":
            self.criterion = nn.CrossEntropyLoss(reduction=reduction)
        else:
            raise ValueError(f"Unsupported loss function: {loss_name}")

    def forward(self, predicted: torch.Tensor, actual: torch.Tensor):
        if isinstance(predicted, torch.Tensor) and isinstance(actual, torch.Tensor):
            return self.criterion(predicted, actual)
        else:
            raise ValueError(
                "Both predicted and actual must be torch.Tensor.".capitalize()
            )


if __name__ == "__main__":
    predicted = torch.tensor(
        [2.5, -1.0, 1.5, -2.0, 2.0, 3.0, -1.5, -0.5], dtype=torch.float
    )
    actual = torch.tensor([1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0], dtype=torch.float)

    criterion = LossFunction(loss_name="BCEWithLogitsLoss", reduction="mean")
    loss = criterion(predicted, actual)

    print("Loss:", loss.item())

### Vision Transformer Model 

In [ ]:
class VisionTransformer(nn.Module):
    def __init__(
        self,
        channels: int = 3,
        patch_size: int = 16,
        image_size: int = 128,
        dimension: int = 256,
        nheads: int = 8,
        num_encoder_layers: int = 6,
        dim_feedforward: int = 1024,
        dropout: float = 0.0,
        layer_norm_eps: float = 1e-05,
        activation: str = "relu",
    ):
        super(VisionTransformer, self).__init__()

        self.channels = channels
        self.patch_size = patch_size
        self.image_size = image_size
        self.dimension = dimension
        self.nheads = nheads
        self.num_encoder_layers = num_encoder_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.layer_norm_eps = layer_norm_eps
        self.activation = activation

        self.patch_embedding = PatchEmbedding(
            channels=channels,
            patch_size=patch_size,
            image_size=image_size,
            dimension=dimension,
        )
        self.transformer_encoder = TransformerEncoder(
            dimension=self.dimension,
            nheads=self.nheads,
            num_encoder_layers=self.num_encoder_layers,
            dim_feedforward=self.dim_feedforward,
            dropout=self.dropout,
            layer_norm_eps=self.layer_norm_eps,
            activation=self.activation,
        )

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            x = self.patch_embedding(x)
            x = self.transformer_encoder(x)
            return x
        else:
            raise ValueError("Input must be a torch.Tensor.".capitalize())

    @staticmethod
    def total_params(model):
        if isinstance(model, VisionTransformer):
            return sum(params.numel() for params in model.parameters())
        else:
            raise ValueError("Input must be a VisionTransformer model.".capitalize())


if __name__ == "__main__":
    image_channels = config_files()["patchEmbeddings"]["channels"]
    image_size = config_files()["patchEmbeddings"]["image_size"]
    patch_size = config_files()["patchEmbeddings"]["patch_size"]
    dimension = config_files()["patchEmbeddings"]["dimension"]
    nheads = config_files()["transfomerEncoderBlock"]["nheads"]
    activation = config_files()["transfomerEncoderBlock"]["activation"]
    dropout = config_files()["transfomerEncoderBlock"]["dropout"]
    num_encoder_layers = config_files()["transfomerEncoderBlock"]["num_encoder_layers"]
    dimension_feedforward = config_files()["transfomerEncoderBlock"][
        "dimension_feedforward"
    ]
    layer_norm_eps = float(config_files()["transfomerEncoderBlock"]["layer_norm_eps"])

    image = torch.randn((1, image_channels, image_size, image_size))

    vision_transformer = VisionTransformer(
        channels=image_channels,
        patch_size=patch_size,
        image_size=image_size,
        dimension=dimension,
        nheads=nheads,
        activation=activation,
        dropout=dropout,
        num_encoder_layers=num_encoder_layers,
        dim_feedforward=dimension_feedforward,
        layer_norm_eps=layer_norm_eps,
    )

    assert vision_transformer(image).size() == (
        1,
        (image_size // patch_size) ** 2,
        dimension,
    )

### Text Transformer

In [ ]:
class TextTransformerEncoder(nn.Module):
    def __init__(
        self,
        dimension: int = 256,
        nheads: int = 8,
        num_encoder_layers: int = 6,
        dim_feedforward: int = 1024,
        dropout: float = 0.0,
        layer_norm_eps: float = 1e-05,
        activation: str = "relu",
    ):
        super(TextTransformerEncoder, self).__init__()
        self.dimension = dimension
        self.nheads = nheads
        self.num_encoder_layers = num_encoder_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.layer_norm_eps = layer_norm_eps
        self.activation = activation

        try:
            self.image_size = config_files()["patchEmbeddings"]["image_size"]
            self.patch_size = config_files()["patchEmbeddings"]["patch_size"]
            self.image_channels = config_files()["patchEmbeddings"]["channels"]
        except KeyError:
            raise ValueError(
                "Image configuration not found in the config files.".capitalize()
            )
        else:
            self.sequence_length = (self.image_size // self.patch_size) ** 2

        if self.dimension is None:
            self.dimension = (self.patch_size**2) * self.image_size

        warnings.warn(
            "The number of vocabularies (unique words) is calculated by multiplying the dimension with the patch size. "
            "If you need a larger vocabulary size, please increase the patch size and image size."
        )

        self.number_of_vocabularies = self.image_size * self.dimension

        self.embedding = nn.Embedding(
            num_embeddings=self.number_of_vocabularies, embedding_dim=self.dimension
        )
        self.transformer_encoder = TransformerEncoder(
            dimension=self.dimension,
            nheads=self.nheads,
            num_encoder_layers=self.num_encoder_layers,
            dim_feedforward=self.dim_feedforward,
            dropout=self.dropout,
            layer_norm_eps=self.layer_norm_eps,
            activation=self.activation,
        )

    def forward(self, x: torch.Tensor):
        if isinstance(x, torch.Tensor):
            x = self.embedding(x)
            x = self.transformer_encoder(x)
            return x
        else:
            raise ValueError("Input must be a torch.Tensor.".capitalize())

    @staticmethod
    def total_params(model):
        if isinstance(model, TextTransformerEncoder):
            return sum(params.numel() for params in model.parameters())
        else:
            raise ValueError(
                "Input must be a TextTransformerEncoder model.".capitalize()
            )


if __name__ == "__main__":
    image_channels = config_files()["patchEmbeddings"]["channels"]
    image_size = config_files()["patchEmbeddings"]["image_size"]
    patch_size = config_files()["patchEmbeddings"]["patch_size"]
    dimension = config_files()["patchEmbeddings"]["dimension"]
    nheads = config_files()["transfomerEncoderBlock"]["nheads"]
    activation = config_files()["transfomerEncoderBlock"]["activation"]
    dropout = config_files()["transfomerEncoderBlock"]["dropout"]
    num_encoder_layers = config_files()["transfomerEncoderBlock"]["num_encoder_layers"]
    dimension_feedforward = config_files()["transfomerEncoderBlock"][
        "dimension_feedforward"
    ]
    layer_norm_eps = float(config_files()["transfomerEncoderBlock"]["layer_norm_eps"])

    sequence_length = (image_size // patch_size) ** 2

    textual_data = torch.randint(0, sequence_length, (1, sequence_length))

    text_transfomer = TextTransformerEncoder(
        dimension=dimension,
        nheads=nheads,
        num_encoder_layers=num_encoder_layers,
        dim_feedforward=dimension_feedforward,
        dropout=dropout,
        layer_norm_eps=layer_norm_eps,
        activation=activation,
    )

    assert (text_transfomer(textual_data).size()) == (1, sequence_length, dimension)

### Multi Modal Classifier (Without Fusion : Do it later (This week))

In [ ]:
class MultiModalClassifier(nn.Module):
    def __init__(
        self,
        channels: int = 3,
        patch_size: int = 16,
        image_size: int = 128,
        dimension: int = 256,
        nheads: int = 8,
        num_encoder_layers: int = 6,
        dim_feedforward: int = 1024,
        dropout: float = 0.1,
        layer_norm_eps: float = 1e-05,
        activation: str = "relu",
    ):
        super(MultiModalClassifier, self).__init__()

        self.channels = channels
        self.patch_size = patch_size
        self.image_size = image_size
        self.dimension = dimension
        self.nheads = nheads
        self.num_encoder_layers = num_encoder_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.layer_norm_eps = layer_norm_eps
        self.activation = activation

        self.vision_transformer = VisionTransformer(
            channels=channels,
            patch_size=patch_size,
            image_size=image_size,
            dimension=dimension,
            nheads=nheads,
            num_encoder_layers=num_encoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            layer_norm_eps=layer_norm_eps,
            activation=activation,
        )

        self.text_transformer = TextTransformerEncoder(
            dimension=self.dimension,
            nheads=self.nheads,
            num_encoder_layers=self.num_encoder_layers,
            dim_feedforward=self.dim_feedforward,
            dropout=self.dropout,
            layer_norm_eps=self.layer_norm_eps,
            activation=self.activation,
        )

    def forward(self, image: torch.Tensor, text: torch.Tensor):
        if isinstance(image, torch.Tensor) and isinstance(text, torch.Tensor):
            image_features = self.vision_transformer(image)
            text_features = self.text_transformer(text)

            return {"image_features": image_features, "text_features": text_features}
        else:
            raise ValueError("Both inputs must be torch.Tensor.".capitalize())


if __name__ == "__main__":
    image_channels = config_files()["patchEmbeddings"]["channels"]
    image_size = config_files()["patchEmbeddings"]["image_size"]
    patch_size = config_files()["patchEmbeddings"]["patch_size"]
    dimension = config_files()["patchEmbeddings"]["dimension"]
    nheads = config_files()["transfomerEncoderBlock"]["nheads"]
    activation = config_files()["transfomerEncoderBlock"]["activation"]
    dropout = config_files()["transfomerEncoderBlock"]["dropout"]
    num_encoder_layers = config_files()["transfomerEncoderBlock"]["num_encoder_layers"]
    dimension_feedforward = config_files()["transfomerEncoderBlock"][
        "dimension_feedforward"
    ]
    layer_norm_eps = float(config_files()["transfomerEncoderBlock"]["layer_norm_eps"])

    number_of_patches = (image_size // patch_size) ** 2
    number_of_sequences = (image_size // patch_size) ** 2

    images = torch.randn(1, image_channels, image_size, image_size)
    texts = torch.randint(0, number_of_sequences, (1, number_of_sequences))

    classifier = MultiModalClassifier(
        channels=image_channels,
        patch_size=patch_size,
        image_size=image_size,
        dimension=dimension,
        nheads=nheads,
        num_encoder_layers=num_encoder_layers,
        dim_feedforward=dimension_feedforward,
        dropout=dropout,
        layer_norm_eps=layer_norm_eps,
        activation=activation,
    )
    output = classifier(image=images, text=texts)
    print(
        "image_features: {}".format(output["image_features"].size()),
        "text features: {}".format(output["text_features"].size()),
    )
    assert (
        output["text_features"].size() == output["image_features"].size()
    ), "Image and text features must be the same size at the end".capitalize()